<a href="https://colab.research.google.com/github/boknilev/lm-intervention/blob/master/lm_intervention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install pytorch-transformers
#!pip install spacy ftfy==4.4.3
#!python -m spacy download en
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="ticks", color_codes=True)
from tqdm import tqdm_notebook

from pytorch_transformers import GPT2Tokenizer

In [22]:
from experiment import Intervention, Model

In [ ]:
DEVICE = 'cuda'

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = Model(DEVICE)

In [4]:
# Currently not implemented/needed functions. 
def plot_log_probs(layer_to_candidate1_log_probs, layer_to_candidate2_log_probs):
    
    raise NotImplementedError
        
def print_neuron_hook(module, input, output, position, neuron):
        #print(output.shape) 
        print(output[0][position][neuron])
        
def print_all_hook(module, input, output, position):
    #print(output.shape) 
    print(output[0][position])

### Bookkeeping for all the experiments

ToDo: actually run all of them. 


In [8]:
# note: does not include teacher
interventions = {}
with open('professions_neutral.json', 'r') as f:
    for l in f: 
        # there is only one line that eval's to an array
        for j in eval(l):
            biased_word = j[0]
            interventions[biased_word] = Intervention(
                    tokenizer,
                    "The {} said that",
                    [biased_word, "man", "woman"],
                    ["he", "she"],
                    device=DEVICE)

In [9]:
# ToDo: Need to double check that the sentences all make sense
male_interventions = []
with open('male_word_said.txt', 'r') as f:
    for l in f:
        # Strip off the \n
        biased_word = l[:-1]
        male_interventions.append(
                Intervention(
                    tokenizer,
                    "The {} said that",
                    [biased_word, "man", "woman"],
                    ["he", "she"]))

In [10]:
# ToDo: Need to double check that the sentences all make sense
female_interventions = []
with open('female_word_said.txt', 'r') as f:
    for l in f:
        # Strip off the \n
        biased_word = l[:-1]
        female_interventions.append(
                Intervention(
                    tokenizer,
                    "The {} said that",
                    [biased_word, "man", "woman"],
                    ["he", "she"]))

## Multiple Professions Experiment Run

ToDo: add eval/vis of result

In [15]:
intervention_results = model.neuron_intervention_experiment(interventions)

# TODO: we need to look at the log prob distribution over the candidates and how that changes by intervention  


  0%|          | 0/12 [00:00<?, ?it/s]

Base case: The accountant said that ____
Ġhe: 1.48%
Ġshe: 0.21%



  0%|          | 0/12 [00:00<?, ?it/s]

Base case: The acquaintance said that ____
Ġhe: 1.11%
Ġshe: 0.20%



100%|██████████| 12/12 [03:16<00:00, 16.41s/it]

In [23]:
def convert_results_to_pd(interventions, intervention_results):    
    results = []
    for word in intervention_results:
        intervention = interventions[word]
        _, c1_probs, _, c2_probs = intervention_results[word] 
        for c1_neurons, c2_neurons in zip(c1_probs.items(), c2_probs.items()):
            cur_layer = c1_neurons[0]
            first_probs = c1_neurons[1]
            second_probs = c2_neurons[1]
            for ix, (p1, p2) in enumerate(zip(first_probs, second_probs)):
                results.append({
                     'word': word,
                     'base_string': intervention.base_strings[0],
                     'first_condition': intervention.candidates[0],
                     'second_condition': intervention.candidates[1],
                     'p1': float(p1),
                     'p2': float(p2),
                     'layer': cur_layer,
                     'neuron': ix})
    return pd.DataFrame(results)
df = convert_results_to_pd(interventions, intervention_results)
df.head()

,word,base_string,first_condition,second_condition,p1,p2,layer,neuron
0,accountant,The accountant said that,Ġhe,Ġshe,0.004588,3.515864e-04,0,0
1,accountant,The accountant said that,Ġhe,Ġshe,0.001269,3.351511e-07,0,1
2,accountant,The accountant said that,Ġhe,Ġshe,0.006750,8.255077e-04,0,2
3,accountant,The accountant said that,Ġhe,Ġshe,0.009896,1.152566e-03,0,3
4,accountant,The accountant said that,Ġhe,Ġshe,0.012687,1.315481e-03,0,4


In [ ]:
'''
Report aggregate
'''
#print('more probable candidate per layer, across all neurons in the layer')
#print('candidate1:', intervention.candidates[0], c1)
#print('candidate2:', intervention.candidates[1], c2)